<a href="https://colab.research.google.com/github/lawun330/FCC-ML-Journey/blob/main/fcc_predict_health_costs_with_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import libraries. You may or may not use all of these.
!pip install -q git+https://github.com/tensorflow/docs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

In [ ]:
# Import data
!wget https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv
dataset = pd.read_csv('insurance.csv')
dataset.tail()

In [ ]:
dataset = dataset.dropna() # drop nan rows if it exists

# convert categorical columns to numeric values
categorical_cols=['sex', 'smoker', 'region']

for categorical_col in categorical_cols:
    dataset[categorical_col] = dataset[categorical_col].map({'male':'male', 'yes': 'smoker', 'northeast':'NE', 'southeast':'SE', 'southwest':'SW', 'northwest':'NW'})
    dataset = pd.get_dummies(dataset, columns=[categorical_col], prefix='', prefix_sep='')

# split the dataset in two
# train_dataset = dataset.sample(frac=0.8, random_state=0) # shuffle and split in one line
train_dataset = dataset[:1070]
test_dataset = dataset.drop(train_dataset.index)

# create feature datasets
train_features = train_dataset.copy()
test_features = test_dataset.copy()

# create labels datasets
train_labels, test_labels = train_features.pop('expenses'), test_features.pop('expenses')

# shuffle train_features for validation data
shuffled_train_dataset = train_dataset.copy().reindex(np.random.permutation(train_dataset.index))
shuffled_train_features = shuffled_train_dataset
shuffled_train_labels = shuffled_train_features.pop('expenses')

In [ ]:
import seaborn as sns
sns.pairplot(train_dataset[['expenses', 'age', 'smoker', 'bmi']], diag_kind='kde')

In [ ]:
# see how different the ranges of each feature
shuffled_train_dataset.describe().transpose()[['mean', 'std']]

In [ ]:
# create normalization layer
normalizer = tf.keras.layers.Normalization(axis=-1)

# pass data to be normalized
shuffled_train_features = tf.convert_to_tensor(shuffled_train_features, dtype = tf.float32)
normalizer.adapt(np.array(shuffled_train_features))

# show how normalization is done
example = np.array(shuffled_train_features[:1])

with np.printoptions(precision=2, suppress=True):
    print('First example:', example)
    print()
    print('Normalized:', normalizer(example).numpy())

In [ ]:
# building linear model
model = tf.keras.Sequential([
    normalizer,
    layers.Dense(2), # having this layer reduces the MAE
    layers.Dense(units=1)
])

model.summary()

In [ ]:
# compiling linear model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.2),
    loss='mean_absolute_error', # loss function
    metrics=['mae', 'mse']) # output mean absolute error and mean squared error

# training the model
# training on : train_features,train_labels vs shuffled_train_features,shuffled_train_labels, yields different results
history = model.fit(shuffled_train_features, shuffled_train_labels, epochs=100, verbose=1,
    # calculate validation results on 20% of the training data.
    validation_split = 0.2)

In [ ]:
# evaluate the model
test_results = {}
test_features = tf.convert_to_tensor(test_features, dtype = tf.float32)
test_labels = tf.convert_to_tensor(test_labels, dtype = tf.float32)
test_results['model'] = model.evaluate(test_features, test_labels, verbose=0)
test_results

In [ ]:
# RUN THIS CELL TO TEST YOUR MODEL. DO NOT MODIFY CONTENTS.
# Test model by checking how well the model generalizes using the test set.
loss, mae, mse = model.evaluate(test_features, test_labels, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} expenses".format(mae))

if mae < 3500:
  print("You passed the challenge. Great job!")
else:
  print("The Mean Abs Error must be less than 3500. Keep trying.")

# Plot predictions.
test_predictions = model.predict(test_features).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True values (expenses)')
plt.ylabel('Predictions (expenses)')
lims = [0, 50000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims,lims)
